In [1]:
from transformers import Qwen2ForCausalLM, Qwen2Model, AutoTokenizer
import torch

torch.set_grad_enabled(False)
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
model = Qwen2ForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map={"": 0},
)
model = model.eval()

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


In [5]:
model.model.embed_tokens.padding_idx

In [2]:
device = "cuda"
tokenizer = AutoTokenizer.from_pretrained(model_name)

prompt = "Give me a short introduction to large language model..1"
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt},
]
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(device)

generated_ids = model.generate(model_inputs.input_ids, max_new_tokens=512)
generated_ids = [
    output_ids[len(input_ids) :]
    for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
response

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


"Okay, so I need to write a short introduction for a large language model. Let me think about how to approach this. First, I should understand what a large language model is. From what I know, it's a type of AI that can understand and generate human language, including speech and text. It's often referred to as LLM, which stands for Large Language Model.\n\nI should start by introducing the concept of LLMs. Maybe mention that they're designed to handle a wide range of tasks like text generation, translation, summarization, etc. I should also highlight their versatility and the various applications they have in different fields.\n\nNext, I should think about the key features of LLMs. They're known for being able to understand vast amounts of data, process it quickly, and produce outputs that are both accurate and contextually relevant. It's important to note their ability to adapt to new information and learn from various datasets, which makes them highly efficient.\n\nI should also tou

In [3]:
model_output_1 = model(
    **model_inputs,
    output_hidden_states=True,
)
model_output_1.hidden_states[-1].shape, model_output_1.hidden_states[-1].dtype

(torch.Size([1, 23, 1536]), torch.bfloat16)

In [ ]:
class Qwen2ModelEmbedPooler(Qwen2ForCausalLM):
    def __init__(self, config):
        super().__init__(config)
        self.model = Qwen2Model(config)
        self.lm_head = None
        self.post_init()

    def forward(self, input_embeds):
        # print(input_embeds.dtype)
        input_embeds = self.model(
            inputs_embeds=input_embeds,
            output_hidden_states=True,
        )[0]
        # print(input_embeds.dtype)
        input_embeds = input_embeds.sum(1) / torch.tensor(
            input_embeds.shape[1],
            device=input_embeds.device,
            dtype=input_embeds.dtype,
        )
        # print(input_embeds.dtype)
        input_embeds = input_embeds.unsqueeze(1)
        return input_embeds


embed_pooler = Qwen2ModelEmbedPooler.from_pretrained(
    "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B", device_map={"": 0}
)
result = embed_pooler(
    # model_output_1.hidden_states[-1],
    torch.cat(
        [
            model_output_1.hidden_states[-1],
            model_output_1.hidden_states[-1],
        ],
        dim=0,
    ),
)
result.shape

torch.Size([2, 1, 1536])

In [ ]:
from transformers import AutoTokenizer, RobertaModel
import torch

model = RobertaModel.from_pretrained("FacebookAI/roberta-base")
tokenizer = AutoTokenizer.from_pretrained("FacebookAI/roberta-base")

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
outputs = model(**inputs)

outputs.last_hidden_state.shape

Some weights of RobertaModel were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


torch.Size([1, 8, 768])

In [5]:
from transformers import RobertaConfig
from transformers import AutoTokenizer, RobertaModel

model_pooler = RobertaModel(
    RobertaConfig(
        # num_hidden_layers=6,
        hidden_size=1536,
    )
)
model_pooler = model_pooler.to("cuda")
# RobertaConfig(
#     num_hidden_layers=6,
# )

In [6]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


count_parameters(model_pooler)

307055616

In [7]:
result = model_pooler(
    # model_output_1.hidden_states[-1],
    inputs_embeds=torch.cat(
        [
            model_output_1.hidden_states[-1],
            model_output_1.hidden_states[-1],
        ],
        dim=0,
    ),
    output_hidden_states=True,
)
result

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 1.1631e-02, -3.3355e-01, -7.0888e-01,  ...,  2.5737e-01,
           8.5082e-01,  9.7829e-01],
         [-6.3120e-02, -9.6038e-01, -1.0395e+00,  ..., -4.8655e-01,
           1.5065e+00,  8.5730e-01],
         [ 4.6764e-01, -9.1962e-01, -1.0514e+00,  ..., -4.7420e-01,
           7.1864e-01,  1.8883e-01],
         ...,
         [ 8.8619e-01, -9.7937e-01, -1.0135e+00,  ...,  5.4751e-02,
           1.3971e+00,  9.3596e-01],
         [ 1.6049e-01, -2.5659e-01, -7.5956e-01,  ...,  2.0967e-01,
           6.3753e-01,  2.2053e-02],
         [ 3.7732e-01, -2.3537e-01, -1.0351e+00,  ...,  1.9910e-01,
           7.2457e-01,  7.0338e-01]],

        [[-1.8583e-04, -1.2355e+00, -2.2558e-01,  ..., -5.9677e-01,
           9.2336e-01,  9.3026e-01],
         [ 6.2141e-01, -3.5414e-01, -1.4218e+00,  ..., -4.0025e-01,
           9.8164e-01,  1.2630e+00],
         [ 5.4169e-01, -1.1041e+00, -1.1333e+00,  ..., -1.4839e+00,
           2.

In [ ]:
input_embeds = result.last_hidden_state  # .shape

input_embeds = input_embeds.sum(1) / torch.tensor(
    input_embeds.shape[1],
    device=input_embeds.device,
    dtype=input_embeds.dtype,
)
# print(input_embeds.dtype)
input_embeds = input_embeds.unsqueeze(1)
input_embeds.shape

torch.Size([2, 1, 1536])

In [8]:
from hidden_capacity_reasoning.models import RobertaModelEmbedPoolerV1
from transformers import RobertaConfig

roberta_model = RobertaModelEmbedPoolerV1(
    RobertaConfig(
        # num_hidden_layers=6,
        hidden_size=1536,
    )
)
roberta_model = roberta_model.to("cuda")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [9]:
result = roberta_model(
    # model_output_1.hidden_states[-1],
    input_embeds=torch.cat(
        [
            model_output_1.hidden_states[-1],
            model_output_1.hidden_states[-1],
        ],
        dim=0,
    ),
    # output_hidden_states=True,
)
result

tensor([[[-1.5889,  0.4153,  1.6604,  ...,  0.9439, -0.3286,  0.7096]],

        [[-1.0837,  0.8477,  1.2010,  ...,  1.1526,  0.4869,  1.1255]]],
       device='cuda:0')

In [10]:
result.shape

torch.Size([2, 1, 1536])

In [11]:
roberta_model.get_input_embeddings()

Embedding(50265, 1536, padding_idx=1)

In [ ]:
from transformers.models.qwen2.modeling_qwen2 import Qwen2RotaryEmbedding, Qwen2Config
from torch import nn

config = Qwen2Config.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")
# config.pad_token_id = tokenizer.pad_token_id
emb = nn.Embedding(config.vocab_size, config.hidden_size, config.pad_token_id).cuda()

new_embeds = emb(model_inputs["input_ids"])

In [5]:
config.pad_token_id

151643

In [24]:
tokenizer.pad_token_id

151643

In [23]:
new_embeds.shape

torch.Size([1, 23, 4096])

In [ ]:
from transformers import Qwen2Config, RobertaConfig, PretrainedConfig


# PretrainedConfig(new_config)
class EmdeddingCompressorConfig(PretrainedConfig):
    roberta: RobertaConfig = RobertaConfig(
        hidden_size=1568,
    )
    qwen: Qwen2Config = Qwen2Config()


config = EmdeddingCompressorConfig()
config.roberta
config.qwen

Qwen2Config {
  "attention_dropout": 0.0,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 22016,
  "max_position_embeddings": 32768,
  "max_window_layers": 28,
  "model_type": "qwen2",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "sliding_window": 4096,
  "tie_word_embeddings": false,
  "transformers_version": "4.49.0",
  "use_cache": true,
  "use_sliding_window": false,
  "vocab_size": 151936
}

In [36]:
config.save_pretrained("EmdeddingCompressorConfig")

In [39]:
EmdeddingCompressorConfig.from_pretrained("EmdeddingCompressorConfig").roberta

RobertaConfig {
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1568,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.49.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 50265
}

In [ ]:
from hidden_capacity_reasoning.models import (
    Qwen2ForCausalLMCompressionV3,
    EmdeddingCompressorConfigV1,
)
from transformers import Qwen2Config, RobertaConfig

# config = EmdeddingCompressorConfigV1()
config = Qwen2Config.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")
config.pooler = RobertaConfig(
    hidden_size=1536,
)
# config.pad_token_id = tokenizer.pad_token_id
# new_model = Qwen2ForCausalLMCompressionV3.from_pretrained(
new_model = Qwen2ForCausalLMCompressionV3.from_pretrained(
    "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
    config=config,
    # torch_dtype=torch.bfloat16,
    # **config,
    # "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
)

Some weights of Qwen2ForCausalLMCompressionV3 were not initialized from the model checkpoint at deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B and are newly initialized: ['embed_pooler.embeddings.LayerNorm.bias', 'embed_pooler.embeddings.LayerNorm.weight', 'embed_pooler.embeddings.position_embeddings.weight', 'embed_pooler.embeddings.token_type_embeddings.weight', 'embed_pooler.embeddings.word_embeddings.weight', 'embed_pooler.encoder.layer.0.attention.output.LayerNorm.bias', 'embed_pooler.encoder.layer.0.attention.output.LayerNorm.weight', 'embed_pooler.encoder.layer.0.attention.output.dense.bias', 'embed_pooler.encoder.layer.0.attention.output.dense.weight', 'embed_pooler.encoder.layer.0.attention.self.key.bias', 'embed_pooler.encoder.layer.0.attention.self.key.weight', 'embed_pooler.encoder.layer.0.attention.self.query.bias', 'embed_pooler.encoder.layer.0.attention.self.query.weight', 'embed_pooler.encoder.layer.0.attention.self.value.bias', 'embed_pooler.encoder.layer.0.attention.self.v

In [8]:
new_model

Qwen2ForCausalLMCompressionV3(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((1536,), eps=1e-06)
    (ro

In [13]:
new_model.load_state_dict(model.state_dict(), strict=False)

_IncompatibleKeys(missing_keys=['embed_pooler.embeddings.word_embeddings.weight', 'embed_pooler.embeddings.position_embeddings.weight', 'embed_pooler.embeddings.token_type_embeddings.weight', 'embed_pooler.embeddings.LayerNorm.weight', 'embed_pooler.embeddings.LayerNorm.bias', 'embed_pooler.encoder.layer.0.attention.self.query.weight', 'embed_pooler.encoder.layer.0.attention.self.query.bias', 'embed_pooler.encoder.layer.0.attention.self.key.weight', 'embed_pooler.encoder.layer.0.attention.self.key.bias', 'embed_pooler.encoder.layer.0.attention.self.value.weight', 'embed_pooler.encoder.layer.0.attention.self.value.bias', 'embed_pooler.encoder.layer.0.attention.output.dense.weight', 'embed_pooler.encoder.layer.0.attention.output.dense.bias', 'embed_pooler.encoder.layer.0.attention.output.LayerNorm.weight', 'embed_pooler.encoder.layer.0.attention.output.LayerNorm.bias', 'embed_pooler.encoder.layer.0.intermediate.dense.weight', 'embed_pooler.encoder.layer.0.intermediate.dense.bias', 'embed

In [ ]:
new_model.save_pretrained("r1_compressor_v4")

: 